In [1]:
import time

log_file = 'log.txt'
# clean the log
f = open(log_file, 'w')
f.close()

def find_repo(term, day):
    import requests
    import math
    from datetime import datetime, timedelta
    from dateutil.relativedelta import relativedelta
    import csv
    import pandas as pd 
    import json
    import os
    
    token=os.environ['GITHUBTOKEN']
    URL = f'https://api.github.com/search/repositories?q={term}+created:SINCE..UNTIL&per_page=100'
    HEADERS = {'Authorization':f'{token}'}
    file_list=[]
    since = datetime.today() - relativedelta(days= day)  # Start fetching repo created {day} days ago
    until = since + timedelta(hours=4) # dividing the total No.of repo into segments of 4 hours each
    fetch_time=f'{until}'.replace(":","_")
    filename = f'{term}_{day}_{fetch_time}.json'
    # clean file
    f= open(filename,'w',encoding='utf-8')
    f.close()
        
    while until < datetime.today():
        day_url = URL.replace('SINCE', since.strftime('%Y-%m-%dT%H:%M:%SZ')).replace('UNTIL', until.strftime('%Y-%m-%dT%H:%M:%SZ'))
        response = requests.get(day_url, headers=HEADERS)
        
        f = open(log_file, 'a')
        f.write(day_url+'\t'+str(response.status_code)+'\n')
        f.close()
        repo_request=response.json()
        
        #saving json files:
        converted_to_string=json.dumps(repo_request)
        f=open(filename,'a',encoding='utf-8')
        f.write(converted_to_string + '\n')
        f.close()
        
        #creating a list of json files'names
        
        # Update dates for the next search
        since = until #move start-date and end-date up 12hours
        until += timedelta(hours=4)
        time.sleep(2)
    file_list.append(filename)
          
    return file_list #return the list of file names

In [2]:
#try the function
file_list = find_repo("tesla",2)

In [3]:
file_list

['tesla_2_2021-10-05 20_54_09.024283.json']

In [5]:
#run the below 
#copy paste the json file below

f=open('tesla_2_2021-10-05 20_54_09.024283.json','r',encoding='utf-8')
content=f.readlines()
content

['{"total_count": 1, "incomplete_results": false, "items": [{"id": 413935043, "node_id": "R_kgDOGKwlww", "name": "a6-Tesla", "full_name": "zzyzzyxx/a6-Tesla", "private": false, "owner": {"login": "zzyzzyxx", "id": 64322145, "node_id": "MDQ6VXNlcjY0MzIyMTQ1", "avatar_url": "https://avatars.githubusercontent.com/u/64322145?v=4", "gravatar_id": "", "url": "https://api.github.com/users/zzyzzyxx", "html_url": "https://github.com/zzyzzyxx", "followers_url": "https://api.github.com/users/zzyzzyxx/followers", "following_url": "https://api.github.com/users/zzyzzyxx/following{/other_user}", "gists_url": "https://api.github.com/users/zzyzzyxx/gists{/gist_id}", "starred_url": "https://api.github.com/users/zzyzzyxx/starred{/owner}{/repo}", "subscriptions_url": "https://api.github.com/users/zzyzzyxx/subscriptions", "organizations_url": "https://api.github.com/users/zzyzzyxx/orgs", "repos_url": "https://api.github.com/users/zzyzzyxx/repos", "events_url": "https://api.github.com/users/zzyzzyxx/events{

In [39]:
#if there is an error-continue in the next line 
repo=[]
import json 
for item in content:
    jsonobj=json.loads(item)
    a=jsonobj["items"]
    
    for b in a:
        id = b.get("id")
        name =b.get("name")
        url = b.get("html_url")
        created = b.get("created_at")
        stars = b.get("stargazers_count")
        watch = b.get("watchers_count")
        language = b.get("language")
        forks = b.get("forks_count")
        
        import requests
        from bs4 import BeautifulSoup
        readme=requests.get(url)
        soup =BeautifulSoup(readme.text,"html.parser")
        readme=soup.find(class_="markdown-body entry-content container-lg")
        if readme == None:
            readme =(" ")
        else: 
            readme=soup.find(class_="markdown-body entry-content container-lg").get_text().replace('\n'," ")
        repo.append({"id": id,
                       "name": name,
                       "url": url,
                       "created": created,
                       "stars": stars,
                       "watch": watch,
                       "language": language,
                       "forks": forks,
                       "readme":readme})

KeyError: 'items'

In [40]:
repo

[{'id': 413935043,
  'name': 'a6-Tesla',
  'url': 'https://github.com/zzyzzyxx/a6-Tesla',
  'created': '2021-10-05T18:27:20Z',
  'stars': 0,
  'watch': 0,
  'language': 'Java',
  'forks': 0,
  'readme': ' '},
 {'id': 414023289,
  'name': 'Tesla-clone',
  'url': 'https://github.com/Raufmaharramov/Tesla-clone',
  'created': '2021-10-06T00:45:16Z',
  'stars': 0,
  'watch': 0,
  'language': 'JavaScript',
  'forks': 0,
  'readme': 'This project was bootstrapped with Create React App, using the Redux and Redux Toolkit template. Available Scripts In the project directory, you can run: npm start Runs the app in the development mode. Open http://localhost:3000 to view it in the browser. The page will reload if you make edits. You will also see any lint errors in the console. npm test Launches the test runner in the interactive watch mode. See the section about running tests for more information. npm run build Builds the app for production to the build folder. It correctly bundles React in produ

In [23]:
#Writing data into .csv file and returning the table:
#error here
import csv
import pandas as pd 
#Writing data into .csv file and returning the table:
with open("repo.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["id", "name", "url", "language", "created", "stars", "watch", "forks","readme"])
    for repo in repo:
        writer.writerow([repo['id'], repo['name'], repo['url'], repo['language'], repo['created'], repo['stars'], repo['watch'], repo['forks'],repo['readme']])
rep = pd.read_csv("repo.csv", delimiter= ";")


In [38]:
repo

IndexError: string index out of range